![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F04+-+scikit-learn&file=04g+-+Vertex+AI+Custom+Model+-+scikit-learn+-+Hyperparameter+Tuning+Job+With+Python+file.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/04%20-%20scikit-learn/04g%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Hyperparameter%20Tuning%20Job%20With%20Python%20file.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F04%2520-%2520scikit-learn%2F04g%2520-%2520Vertex%2520AI%2520Custom%2520Model%2520-%2520scikit-learn%2520-%2520Hyperparameter%2520Tuning%2520Job%2520With%2520Python%2520file.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/04%20-%20scikit-learn/04g%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Hyperparameter%20Tuning%20Job%20With%20Python%20file.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/04%20-%20scikit-learn/04g%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Hyperparameter%20Tuning%20Job%20With%20Python%20file.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# 04g - Vertex AI > Training > Hyperparameter Tuning Jobs

**04 Series Overview**

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) notebook, the model training happened directly in the notebook.  The model was then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `04a-04i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI Training as managed jobs.  These jobs will be demonstrated as:

-  Custom Job that trains and saves (to GCS) a model from a python script (`04a`), python source distribution (`04b`), and custom container (`04c`)
-  Training Pipeline that trains and registers a model from a python script (`04d`), python source distribution (`04e`), and custom container (`04f`)
-  Hyperparameter Tuning Jobs from a python script (`04g`), python source distribution (`04h`), and custom container (`04i`)

**This Notebook (`04g`): An extension of `04a` with Hyperparameter Tuning - And Tensorboard HParams**

This notebook trains the same scikit-learn Keras model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a Python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb). 

The script is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service. 

The Custom Job is then used as the input for a Vertex AI > Training > Hyperparameter Tuning Job.  This runs and manages the tuning loops for the number of trials in each loop, collects the metric(s) and manages the parameters with the selected search algorithm for parameter modification.

This job is launched using the Vertex AI client library:
- [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference)
    - [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)
        - [`aiplatform` package](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
            - Custom Job:
                - [`aiplatform.CustomJob.from_local_script()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.CustomJob#google_cloud_aiplatform_CustomJob_from_local_script)
            - Hyperparameter Tuning Job:
                - [`aiplatform.HyperparameterTuningJob()`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.HyperparameterTuningJob#google_cloud_aiplatform_HyperparameterTuningJob)

**Vertex AI Training**

In Vertex AI Training you can run your training code as a job where you specify the compute resources to use. For tips on preparing code, running training jobs, and workflows for building custom containers with software and training code combined please visit these [tips notebooks](../Tips/readme.md) in this repository:
- [Python Packages](../Tips/Python%20Packages.ipynb)
- [Python Custom Containers](../Tips/Python%20Custom%20Containers.ipynb)
- [Python Training](../Tips/Python%20Training.ipynb)

<p align="center" width="100%">
    <img src="../architectures/overview/training.png" width="45%">
    &nbsp; &nbsp; &nbsp; &nbsp;
    <img src="../architectures/overview/training2.png" width="45%">
</p>

**Prerequisites:**
-  [01 - BigQuery - Table Data Source](../01%20-%20Data%20Sources/01%20-%20BigQuery%20-%20Table%20Data%20Source.ipynb)
-  Understanding:
    -  Model overview in [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb)
    -  Convert notebook code to Python Script in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Script](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Script.ipynb)

**Resources:**
-  [sklearn](https://scikit-learn.org/stable/)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
-  Vertex AI Hyperparameter Tuning
   -  [Overview of Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)
   -  [Using Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05g_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05g_console.png" width="45%">
</p>

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [2]:
REGION = 'us-central1'
EXPERIMENT = '04g'
SERIES = '04'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id-splits' + '-' + VAR_TARGET # add more variables to the string with space delimiters
PENALTY = 'l2'


packages:

In [1]:
from google.cloud import aiplatform
from datetime import datetime
import pkg_resources
from IPython.display import Markdown as md
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [6]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'mg-ce-demos-main@mg-ce-demos.iam.gserviceaccount.com'

List the service accounts current roles:

In [7]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/aiplatform.admin
roles/bigquery.admin
roles/editor
roles/storage.objectAdmin


>Note: If the resulting list is missing [roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles) then [revisit the setup notebook](../00%20-%20Setup/00%20-%20Environment%20Setup.ipynb#permissions) and add this permission to the service account with the provided instructions.

environment:

In [8]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [9]:
FRAMEWORK = 'sklearn'
TASK = 'classification'
MODEL_TYPE = 'logistric-regression'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [10]:
aiplatform.init(experiment = EXPERIMENT_NAME)

---
## Training

### Python File for Training

This notebook trains the same scikit-learn Keras model from [04 - Vertex AI Custom Model - scikit-learn - in Notebook](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20in%20Notebook.ipynb) by first modifying and saving the training code to a python script as shown in [04 - Vertex AI Custom Model - scikit-learn - Notebook to Hyperparameter Tuning Script.ipynb](./04%20-%20Vertex%20AI%20Custom%20Model%20-%20scikit-learn%20-%20Notebook%20to%20Hyperparameter%20Tuning%20Script.ipynb) which stores the script in [`./code/hp_train.py`](./code/hp_train.py).

**Review the script:**

In [2]:
SCRIPT_PATH = './code/hp_train.py'

with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python


# package import
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

import pickle
import pandas as pd 
import numpy as np 

from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud import storage
import argparse
import os
import sys
import hypertune

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--penalty', dest = 'penalty', default = 'l2', type = str, help = 'Penalty term')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str)
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--bq_project', dest = 'bq_project', type=str)
parser.add_argument('--bq_dataset', dest = 'bq_dataset', type=str)
parser.add_argument('--bq_table', dest = 'bq_table', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest = 'experiment', type=str)
parser.add_argument('--series', dest = 'series', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)

# hyperparameters
parser.add_argument('--solver', dest = 'solver', type = str, help = 'Logistic regression solver')
parser.add_argument('--C', dest = 'C', type=float)
args = parser.parse_args()

# Model Training
VAR_TARGET = str(args.var_target)
VAR_OMIT = str(args.var_omit).split('-')

# clients
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)
hpt = hypertune.HyperTune()
args.run_name = f'{args.run_name}-{hpt.trial_id}'

# Vertex AI Experiment
if args.run_name in [run.name for run in aiplatform.ExperimentRun.list(experiment = args.experiment_name)]:
    expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)
else:
    expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment': args.experiment, 'series': args.series, 'project_id': args.project_id})
expRun.log_params({'hyperparameter.C': args.C})

# get schema from bigquery source
query = f"SELECT * FROM `{args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS` WHERE TABLE_NAME = '{args.bq_table}'"
schema = bq.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM `{args.bq_project}.{args.bq_dataset}.{args.bq_table}` WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}', 'nclasses': nclasses, 'var_split': 'splits', 'var_target': VAR_TARGET})

train_query = f"SELECT * FROM `{args.bq_project}.{args.bq_dataset}.{args.bq_table}` WHERE splits = 'TRAIN'"
train = bq.query(train_query).to_dataframe()
X_train = train.loc[:, ~train.columns.isin(VAR_OMIT)]
y_train = train[VAR_TARGET].astype('int')

val_query = f"SELECT * FROM `{args.bq_project}.{args.bq_dataset}.{args.bq_table}` WHERE splits = 'VALIDATE'"
val = bq.query(val_query).to_dataframe()
X_val = val.loc[:, ~val.columns.isin(VAR_OMIT)]
y_val = val[VAR_TARGET].astype('int')

test_query = f"SELECT * FROM `{args.bq_project}.{args.bq_dataset}.{args.bq_table}` WHERE splits = 'TEST'"
test = bq.query(test_query).to_dataframe()
X_test = test.loc[:, ~test.columns.isin(VAR_OMIT)]
y_test = test[VAR_TARGET].astype('int')

# Logistic Regression
# instantiate the model 
logistic = LogisticRegression(solver=args.solver, penalty=args.penalty, C=args.C)

# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

expRun.log_params({'solver': args.solver, 'penalty': args.penalty, 'C':args.C})

# define pipeline
pipe = Pipeline(steps=[("scaler", scaler), ("logistic", logistic)])

# define grid search model
model = pipe.fit(X_train, y_train)

# test evaluations:
y_pred = model.predict(X_test)
test_acc = metrics.accuracy_score(y_test, y_pred) 
test_prec = metrics.precision_score(y_test, y_pred)
test_rec = metrics.recall_score(y_test, y_pred)
test_rocauc = metrics.roc_auc_score(y_test, y_pred)
expRun.log_metrics({'test_accuracy': test_acc, 'test_precision': test_prec, 'test_recall': test_rec, 'test_roc_auc': test_rocauc})

# val evaluations:
y_pred_val = model.predict(X_val)
val_acc = metrics.accuracy_score(y_val, y_pred_val) 
val_prec = metrics.precision_score(y_val, y_pred_val)
val_rec = metrics.recall_score(y_val, y_pred_val)
val_rocauc = metrics.roc_auc_score(y_val, y_pred_val)
expRun.log_metrics({'validation_accuracy': val_acc, 'validation_precision': val_prec, 'validation_recall': val_rec, 'validation_roc_auc': val_rocauc})

# training evaluations:
y_pred_training = model.predict(X_train)
training_acc = metrics.accuracy_score(y_train, y_pred_training) 
training_prec = metrics.precision_score(y_train, y_pred_training)
training_rec = metrics.recall_score(y_train, y_pred_training)
training_rocauc = metrics.roc_auc_score(y_train, y_pred_training)
expRun.log_metrics({'training_accuracy': training_acc, 'training_precision':training_prec, 'training_recall': training_rec, 'training_roc_auc': training_rocauc})

# report hypertune info back to Vertex AI Training > Hyperparamter Tuning Job
hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag = 'roc_auc',
    metric_value = val_rocauc)

file_name = 'model.pkl'

# Use predefined environment variable to establish model directory
model_directory = os.environ['AIP_MODEL_DIR']
storage_path = f'/gcs/{model_directory[5:]}' + file_name
os.makedirs(os.path.dirname(storage_path), exist_ok=True)

# output the model save files directly to GCS destination
with open(storage_path,'wb') as f:
    pickle.dump(model,f)

expRun.log_params({'model.save': storage_path})
expRun.end_run()

```

### Setup Training Job

In [12]:
CMDARGS = [
    "--penalty=" + PENALTY,
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME
]

In [13]:
customJob = aiplatform.CustomJob.from_local_script(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    script_path = SCRIPT_PATH,
    container_uri = TRAIN_IMAGE,
    args = CMDARGS,
    requirements = ['google-cloud-aiplatform', 'protobuf',  'db-dtypes>=1.0.0', 'google-auth>=2.6.0', 'google-cloud-bigquery>=3.0.1'],
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

Training script copied to:
gs://mg-ce-demos/04/04g/models/20230309095759/aiplatform-2023-03-09-09:58:03.730-aiplatform_custom_trainer_script-0.1.tar.gz.


### Setup Hyperparameter Tuning Job

In [14]:
METRIC_SPEC = {
    "roc_auc": "maximize"
}

PARAMETER_SPEC = {
    "solver":aiplatform.hyperparameter_tuning.CategoricalParameterSpec(values=['newton-cg', 'liblinear']),
    "C": aiplatform.hyperparameter_tuning.DiscreteParameterSpec(values=[0.01, 0.1, 1.0], scale='linear')
}


In [15]:
tuningJob = aiplatform.HyperparameterTuningJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    custom_job = customJob,
    metric_spec = METRIC_SPEC,
    parameter_spec = PARAMETER_SPEC,
    max_trial_count = 10,
    parallel_trial_count = 3,
    search_algorithm = None,
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job

In [16]:
tuningJob.run(
    service_account = SERVICE_ACCOUNT
)

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/633472233130/locations/us-central1/hyperparameterTuningJobs/9091916213344272384
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/633472233130/locations/us-central1/hyperparameterTuningJobs/9091916213344272384')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/9091916213344272384?project=633472233130
HyperparameterTuningJob projects/633472233130/locations/us-central1/hyperparameterTuningJobs/9091916213344272384 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/633472233130/locations/us-central1/hyperparameterTuningJobs/9091916213344272384 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/633472233130/locations/us-central1/hyperparameterTuningJobs/9091916213344272384 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJ

In [17]:
tuningJob.resource_name, tuningJob.display_name

('projects/633472233130/locations/us-central1/hyperparameterTuningJobs/9091916213344272384',
 '04_04g_20230309095759')

Create hyperlinks to job and tensorboard here:

In [18]:
job_link = f"https://console.cloud.google.com/ai/platform/locations/{REGION}/training/{tuningJob.resource_name.split('/')[-1]}?project={PROJECT_ID}"

print(f'Review the Job here:\n{job_link}')

Review the Job here:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/9091916213344272384?project=mg-ce-demos


### Get Best Run

In [19]:
# if trial.state.name == 'SUCCEEDED'
auc_roc = [trial.final_measurement.metrics[0].value if trial.state.name == 'SUCCEEDED' else 1 for trial in tuningJob.trials]
auc_roc

[0.764617838683281,
 0.764617838683281,
 0.7450276042799582,
 0.7744217602519086,
 0.7450276042799582,
 0.7450276042799582,
 0.7744217602519086,
 0.764617838683281,
 0.764617838683281,
 0.764617838683281]

In [20]:
best = tuningJob.trials[auc_roc.index(max(auc_roc))]
best

id: "4"
state: SUCCEEDED
parameters {
  parameter_id: "C"
  value {
    number_value: 1.0
  }
}
parameters {
  parameter_id: "solver"
  value {
    string_value: "newton-cg"
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "roc_auc"
    value: 0.7744217602519086
  }
}
start_time {
  seconds: 1678374447
  nanos: 397917952
}
end_time {
  seconds: 1678374769
}

In [21]:
best.id

'4'

---
## Serving

### Upload The Model
Only upload the best model from the hyperparmeter tuning job:

In [22]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if f'{RUN_NAME}-{best.id}' in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/{best.id}/model",
        is_default_version = True,
        version_aliases = [f'{RUN_NAME}-{best.id}'],
        version_description = f'{RUN_NAME}-{best.id}',
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}-{best.id}'}        
    )

Model Already in Registry:
Loading model as new default version.
Creating Model
Create Model backing LRO: projects/633472233130/locations/us-central1/models/model_04_04g/operations/9072749732807311360
Model created. Resource name: projects/633472233130/locations/us-central1/models/model_04_04g@2
To use this Model in another session:
model = aiplatform.Model('projects/633472233130/locations/us-central1/models/model_04_04g@2')


>**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`
>
>**Retrieve a Model Resource**
>[aiplatform.Model()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
>```Python
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

In [23]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_04_04g?project=mg-ce-demos


### Vertex AI Experiment Update and Review

In [24]:
expRun = aiplatform.ExperimentRun(run_name = f'{RUN_NAME}-{best.id}', experiment = EXPERIMENT_NAME)

In [25]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'hyperparameterTuningJobs.display_name': tuningJob.display_name,
    'hyperparameterTuning.resource_name': tuningJob.resource_name,
    'hyperparameterTuning.link': job_link
})

Complete the experiment run:

In [26]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Need to add the `hyperparameterTuning` job information to each run of the experiment:

In [27]:
for trial in tuningJob.trials:
    expRun = aiplatform.ExperimentRun(run_name = f'{RUN_NAME}-{trial.id}', experiment = EXPERIMENT_NAME)
    expRun.log_params({
        'hyperparameterTuningJobs.display_name': tuningJob.display_name,
        'hyperparameterTuning.resource_name': tuningJob.resource_name,
        'hyperparameterTuning.link': job_link
    })
    expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [28]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [29]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.hyperparameterTuning.resource_name,param.data_source,param.model.save,param.C,param.var_target,param.nclasses,...,metric.training_accuracy,metric.test_recall,metric.training_precision,metric.validation_accuracy,metric.test_roc_auc,metric.validation_recall,metric.training_roc_auc,metric.training_recall,metric.validation_precision,metric.test_precision
0,experiment-04-04g-sklearn-classification-logis...,run-20230309095759-10,system.ExperimentRun,COMPLETE,projects/633472233130/locations/us-central1/hy...,bq://mg-ce-demos.fraud.fraud_prepped,/gcs/mg-ce-demos/04/04g/models/20230309095759/...,0.1,Class,2.0,...,0.999197,0.607143,0.876866,0.998981,0.803519,0.529412,0.805122,0.610390,0.843750,0.918919
1,experiment-04-04g-sklearn-classification-logis...,run-20230309095759-9,system.ExperimentRun,COMPLETE,projects/633472233130/locations/us-central1/hy...,bq://mg-ce-demos.fraud.fraud_prepped,/gcs/mg-ce-demos/04/04g/models/20230309095759/...,1.0,Class,2.0,...,0.999206,0.607143,0.877778,0.998981,0.803519,0.529412,0.807720,0.615584,0.843750,0.918919
2,experiment-04-04g-sklearn-classification-logis...,run-20230309095759-8,system.ExperimentRun,COMPLETE,projects/633472233130/locations/us-central1/hy...,bq://mg-ce-demos.fraud.fraud_prepped,/gcs/mg-ce-demos/04/04g/models/20230309095759/...,0.1,Class,2.0,...,0.999197,0.607143,0.876866,0.998981,0.803519,0.529412,0.805122,0.610390,0.843750,0.918919
3,experiment-04-04g-sklearn-classification-logis...,run-20230309095759-7,system.ExperimentRun,COMPLETE,projects/633472233130/locations/us-central1/hy...,bq://mg-ce-demos.fraud.fraud_prepped,/gcs/mg-ce-demos/04/04g/models/20230309095759/...,1.0,Class,2.0,...,0.999210,0.607143,0.878229,0.999016,0.803501,0.549020,0.809018,0.618182,0.848485,0.894737
4,experiment-04-04g-sklearn-classification-logis...,run-20230309095759-6,system.ExperimentRun,COMPLETE,projects/633472233130/locations/us-central1/hy...,bq://mg-ce-demos.fraud.fraud_prepped,/gcs/mg-ce-demos/04/04g/models/20230309095759/...,0.01,Class,2.0,...,0.999166,0.607143,0.879377,0.998945,0.803536,0.490196,0.793438,0.587013,0.862069,0.944444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,experiment-04-04g-sklearn-classification-logis...,run-20230215154055-4,system.ExperimentRun,RUNNING,NaN,bq://mg-ce-demos.fraud.fraud_prepped,NaN,NaN,Class,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,experiment-04-04g-sklearn-classification-logis...,run-20230215154055-3,system.ExperimentRun,RUNNING,NaN,bq://mg-ce-demos.fraud.fraud_prepped,NaN,NaN,Class,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,experiment-04-04g-sklearn-classification-logis...,run-20230215154055-2,system.ExperimentRun,RUNNING,NaN,bq://mg-ce-demos.fraud.fraud_prepped,NaN,NaN,Class,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,experiment-04-04g-sklearn-classification-logis...,run-20230215154055-5,system.ExperimentRun,RUNNING,NaN,bq://mg-ce-demos.fraud.fraud_prepped,NaN,NaN,Class,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Review Experiment and Run in Console

In [30]:
print(f'Review The Experiment in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/experiments/{EXPERIMENT_NAME}?project={PROJECT_ID}')

Review The Experiment in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/experiments/experiment-04-04g-sklearn-classification-logistric-regression?project=mg-ce-demos


### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [31]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [32]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [33]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-04-04h-sklearn-classification-logistric-regression
experiment-04-04g-sklearn-classification-logistric-regression
experiment-04-04f-sklearn-classification-logistric-regression
experiment-04-04e-sklearn-classification-logistric-regression
experiment-04-04d-sklearn-classification-logistric-regression
experiment-04-04-sklearn-classification-logistic-regression
experiment-04-04c-sklearn-classification-logistric-regression
experiment-04-04b-sklearn-classification-logistric-regression
experiment-04-04a-sklearn-classification-logistric-regression
experiment-04-04-sklearn-classification-sklearn-logistic-regression
experiment-04-04-tf-classification-sklearn-logistic-regression


Create ranks for models within experiment and across the entire SERIES:

In [34]:
def ranker(metric = 'metric.test_roc_auc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks = ranks[~ranks['param.model.display_name'].isnull()]
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_roc_auc')
ranks

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_roc_auc,series_rank,experiment_rank
138,experiment-04-04-sklearn-classification-logist...,run-20221102155936,04_04,2,0.803501,1.0,1.0
137,experiment-04-04-sklearn-classification-logist...,run-20221104083407,04_04,3,0.803501,1.0,1.0
136,experiment-04-04-sklearn-classification-logist...,run-20221104090606,04_04,4,0.803501,1.0,1.0
135,experiment-04-04-sklearn-classification-logist...,run-20221107104554,04_04,5,0.803501,1.0,1.0
133,experiment-04-04-sklearn-classification-logist...,run-20221108150830,04_04,6,0.803501,1.0,1.0
130,experiment-04-04-sklearn-classification-logist...,run-20230109082541,04_04,7,0.803501,1.0,1.0
129,experiment-04-04-sklearn-classification-logist...,run-20230109085605,04_04,8,0.803501,1.0,1.0
128,experiment-04-04-sklearn-classification-logist...,run-20230109092136,04_04,9,0.803501,1.0,1.0
127,experiment-04-04-sklearn-classification-logist...,run-20230109101026,04_04,10,0.803501,1.0,1.0
126,experiment-04-04-sklearn-classification-logist...,run-20230109115259,04_04,11,0.803501,1.0,1.0


In [35]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_roc_auc,series_rank,experiment_rank
16,experiment-04-04g-sklearn-classification-logis...,run-20230309095759-4,04_04g,2,0.803501,1.0,1.0


In [36]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

The current model is ranked 1.0 within this experiment and 1.0 across this series.


### Create/Retrieve The Endpoint For This Series

In [37]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Endpoint Exists: projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/1476393427452035072?project=mg-ce-demos


In [38]:
endpoint.display_name

'04'

In [39]:
endpoint.traffic_split

{'7621915758812987392': 100}

In [40]:
deployed_models = endpoint.list_models()
deployed_models

[id: "7621915758812987392"
 model: "projects/633472233130/locations/us-central1/models/model_04_04f"
 display_name: "04_04f"
 create_time {
   seconds: 1678372948
   nanos: 575042000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "4"]

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [41]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deploy = True
    print('No models currently deployed.')

The current model is ranked better (1.0) than a currently deployed model (1.0).


### Deploy Model To Endpoint

In [42]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})')

Deploying model with 100% of traffic...
Deploying Model projects/633472233130/locations/us-central1/models/model_04_04g to Endpoint : projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Deploy Endpoint model backing LRO: projects/633472233130/locations/us-central1/endpoints/1476393427452035072/operations/7546029459128713216
Endpoint model deployed. Resource name: projects/633472233130/locations/us-central1/endpoints/1476393427452035072


### Remove Deployed Models without Traffic

In [43]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

Model 04_04g with version 2 has traffic = 100
Undeploying Endpoint model: projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Undeploy Endpoint model backing LRO: projects/633472233130/locations/us-central1/endpoints/1476393427452035072/operations/6194949570917564416
Endpoint model undeployed. Resource name: projects/633472233130/locations/us-central1/endpoints/1476393427452035072
Undeploying 04_04f with version 4 because it has no traffic.


In [44]:
endpoint.traffic_split

{'1682793750218145792': 100}

In [45]:
endpoint.list_models()

[id: "1682793750218145792"
 model: "projects/633472233130/locations/us-central1/models/model_04_04g"
 display_name: "04_04g"
 create_time {
   seconds: 1678376185
   nanos: 933716000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "2"]

---
## Prediction

See many more details on requesting predictions in the [04Tools - Prediction](./04Tools%20-%20Prediction.ipynb) notebook.

### Prepare a record for prediction: instance and parameters lists

In [46]:
n = 10
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({VAR_OMIT.replace('-', ',')})
        FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [47]:
pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,15689,-0.745436,1.208740,2.418708,3.033260,-0.501836,1.497792,-0.848100,0.999515,0.376560,...,-0.149698,-0.066289,0.035959,-0.088377,-0.395340,-0.422684,0.188265,0.020712,0.045274,0.0
1,51457,-5.194546,-1.741200,2.409787,3.979815,4.745377,-0.470931,-6.630054,-1.774991,0.541608,...,-1.728441,-1.909447,0.063879,-5.298245,-0.242950,-1.611578,0.186398,0.421281,0.010040,0.0
2,152527,2.019771,0.058431,-1.041463,1.626049,0.227237,-0.196559,-0.130095,-0.015467,-0.944086,...,-0.186591,0.191799,0.445460,0.204942,0.859635,-0.411058,2.328281,-0.226044,-0.090710,0.0
3,134025,2.101359,-0.885248,-0.019706,-0.810424,-1.429294,-0.832277,-1.110466,-0.093680,-0.347834,...,0.022980,0.493235,1.524276,0.188728,0.619306,-0.262769,-0.123342,0.023263,-0.048398,0.0
4,161647,-0.484457,1.664356,1.947575,4.371354,0.613689,0.689506,0.536490,-0.021216,-2.204157,...,0.242229,0.374826,1.234508,-0.614699,0.000011,0.583962,0.825513,-0.023025,0.073964,0.0
5,40874,1.152656,0.041226,0.092043,0.464861,0.107094,0.242897,-0.031947,0.167831,-0.077975,...,-0.193854,0.037042,0.296369,-0.057350,-0.228960,0.481854,0.597416,-0.023728,-0.016767,0.0
6,153526,2.112608,0.134436,-2.485294,0.319255,0.901347,-1.135053,0.806315,-0.388994,-0.281632,...,-0.288548,0.302005,0.952619,-0.139831,0.838490,0.634229,0.670702,-0.138295,-0.098389,0.0
7,169454,1.873996,0.218019,0.305808,3.896433,-0.447078,0.196240,-0.598065,0.163851,-0.109903,...,-0.356956,0.158820,0.542595,0.205643,-0.004049,-0.268981,0.029799,0.011302,-0.033850,0.0
8,8748,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.195556,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0
9,45652,-18.328663,-25.516989,1.920922,5.635984,24.655965,-16.057255,-16.543258,2.322645,1.845834,...,6.405990,1.981090,-1.783628,4.302034,-1.409108,1.895257,0.549036,-0.947777,0.526315,0.0


In [48]:
newobs = pred.to_dict(orient = 'split')['data']
#newobs[0]

In [49]:
newobs[0]

[15689,
 -0.745436078271323,
 1.2087397736926002,
 2.4187082268938904,
 3.0332596368148197,
 -0.501835970393595,
 1.4977918233493501,
 -0.848099888253166,
 0.999514781958054,
 0.37655997467011204,
 0.0293332716957848,
 1.18623426620899,
 -2.8173544111887403,
 0.0961888675988831,
 1.88716881651866,
 -0.583405886893678,
 0.0421613714076143,
 1.1301829871081899,
 0.8452847141011891,
 0.836468961167615,
 -0.14969799033976003,
 -0.0662889632152991,
 0.035959384223694,
 -0.08837740606587301,
 -0.39533962104803294,
 -0.42268367941540896,
 0.18826450465864397,
 0.0207119544421035,
 0.04527401782766201,
 0.0]

In [50]:
#instances = [json_format.ParseDict(newobs[0], Value())]

### Get Predictions: Python Client

In [51]:
prediction = endpoint.predict(instances = newobs[0:1])
prediction

Prediction(predictions=[0.0], deployed_model_id='1682793750218145792', model_version_id='2', model_resource_name='projects/633472233130/locations/us-central1/models/model_04_04g', explanations=None)

In [52]:
prediction = endpoint.predict(instances = newobs)
prediction

Prediction(predictions=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], deployed_model_id='1682793750218145792', model_version_id='2', model_resource_name='projects/633472233130/locations/us-central1/models/model_04_04g', explanations=None)

In [53]:
prediction.predictions[0]

0.0

In [54]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [55]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": newobs[0:1]}))

In [56]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    0
  ],
  "deployedModelId": "1682793750218145792",
  "model": "projects/633472233130/locations/us-central1/models/model_04_04g",
  "modelDisplayName": "04_04g",
  "modelVersionId": "2"
}


### Get Predictions: gcloud (CLI)

In [57]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[0]


---
## Remove Resources
see notebook "99 - Cleanup"